In [1]:
import os
import shutil
import random
from pathlib import Path

# 定义路径和参数
DATA_DIR = "dataset/"
OUTPUT_DIR = "dataset/output/"
TRAIN_DIR = os.path.join(OUTPUT_DIR, "train")
TEST_DIR = os.path.join(OUTPUT_DIR, "test")
ANNOTATIONS_DIR = os.path.join(DATA_DIR, "annotations")
IMAGES_DIR = os.path.join(DATA_DIR, "images")
VOC_YAML_PATH ="voc.yaml"
DATASET_URL = "andrewmvd/face-mask-detection"
TRAIN_RATIO = 0.8

# 下载并解压 Kaggle 数据集
def download_and_extract(url, output_dir):
    try:
        import kagglehub
    except ImportError:
        print("❌ 未安装 kagglehub 库，请先安装：pip install kagglehub")
        return

    if os.path.exists(output_dir):
        print(f"✅ 数据集已存在，取消下载：{os.path.abspath(output_dir)}")
        return

    print("🚀 开始下载数据集...")
    url_path = kagglehub.dataset_download(url)
    shutil.copytree(url_path, output_dir)
    shutil.rmtree(url_path)
    print(f"✅ 数据集已成功下载到：{os.path.abspath(output_dir)}")

# 划分数据集并复制文件
def split_dataset(train_ratio):
    # 创建输出目录
    if os.path.exists(OUTPUT_DIR):
        print(f"✅ 输出目录已存在，取消划分：{os.path.abspath(OUTPUT_DIR)}")
    else:
        os.makedirs(TRAIN_DIR, exist_ok=True)
        os.makedirs(TEST_DIR, exist_ok=True)
        # 获取所有文件名
        annotation_files = list(Path(ANNOTATIONS_DIR).glob("*.xml"))
        image_files = list(Path(IMAGES_DIR).glob("*.png"))  # 假设图片格式为 .png

        # 确保注释文件和图片文件数量一致
        assert len(annotation_files) == len(image_files), "注释文件和图片文件数量不一致"

        # 打乱顺序
        combined = list(zip(annotation_files, image_files))
        random.shuffle(combined)
        annotation_files[:], image_files[:] = zip(*combined)

        # 划分训练集和测试集
        split_index = int(len(annotation_files) * train_ratio)
        train_annotations = annotation_files[:split_index]
        train_images = image_files[:split_index]
        test_annotations = annotation_files[split_index:]
        test_images = image_files[split_index:]

        # 复制文件到训练集和测试集目录
        def copy_files(file_list, target_dir):
            for file in file_list:
                shutil.copy(file, target_dir)

        copy_files(train_annotations, TRAIN_DIR)
        copy_files(train_images, TRAIN_DIR)
        copy_files(test_annotations, TEST_DIR)
        copy_files(test_images, TEST_DIR)

        print(f"✅ 数据集划分完成，训练集路径：{TRAIN_DIR}")
        print(f"✅ 数据集划分完成，测试集路径：{TEST_DIR}")
        with open(VOC_YAML_PATH, "w") as f:
            f.write(f"train: {os.path.abspath(TRAIN_DIR)}\n")
            f.write(f"val: {os.path.abspath(TEST_DIR)}\n")
            f.write("\n")
            f.write("nc: 2\n")  # 假设有两个类别，例如“戴口罩”和“未戴口罩”
            f.write("names: ['with_mask', 'without_mask']\n")  # 类别名称
        print(f"✅ voc.yaml 文件已生成：{VOC_YAML_PATH}")

# 生成 voc.yaml 文件


# 下载数据集
download_and_extract(DATASET_URL, DATA_DIR)# 划分数据集
split_dataset(TRAIN_RATIO)# 生成 voc.yaml 文件

🚀 开始下载数据集...


100%|██████████| 398M/398M [00:05<00:00, 71.3MB/s]

Extracting files...


✅ 数据集已成功下载到：/content/dataset
✅ 数据集划分完成，训练集路径：dataset/output/train
✅ 数据集划分完成，测试集路径：dataset/output/test
✅ voc.yaml 文件已生成：voc.yaml


In [ ]:
!pip install ultralytics
!nvidia-smi
from ultralytics import YOLO
device = 'cpu'  # 使用GPU训练,可选cuda或cpu

model = YOLO("baseModel/yolov8n.pt")  # 使用预训练模型
model.train(
    data="voc.yaml",
    device=0 if device == "cuda" else "cpu",
    epochs=30,
    batch=64,
    imgsz=640,
    optimizer="AdamW",
    augment=True,
    lr0=0.001,               # 适当提高初始学习率
    lrf=0.01,                # 添加余弦退火最终学习率
    amp=True,               # 保持混合精度训练
    pretrained=True,        # 确保使用预训练权重
    save=True,  # 保存模型
    exist_ok=True,
    )  # 训练模型

/bin/bash: line 1: nvidia-smi: command not found
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=baseModel/yolov8n.pt, data=voc.yaml, epochs=30, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

train: Scanning /content/dataset/output/train.cache... 0 images, 682 backgrounds, 0 corrupt: 100%|██████████| 682/682 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/dataset/output/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 73.9±29.1 MB/s, size: 415.9 KB)


val: Scanning /content/dataset/output/test.cache... 0 images, 171 backgrounds, 0 corrupt: 100%|██████████| 171/171 [00:00<?, ?it/s]

WARNING ⚠️ No labels found in /content/dataset/output/test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/detect/train/labels.jpg... 
zero-size array to reduction operation maximum which has no identity


optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/11 [00:00<?, ?it/s]